# PHE SEIR Model Sensitivity Analysis

In this notebook we present how to use the `epimodels` module to set up a parameter sensitivity analysis of the model built by Public Health England in collaboration with University of Cambridge, using baseline contact matrices. We aim to explore the behaviour of the output of the model for changes in key parameters. This is done to identify those parameters we can infer in the model.

The analyses are run for:
 - Dates: **15 Feb 2020** - **15 May 2020**, using toy data;
 - PHE regions of interest: **London**.

We use synthetically generated serology and mortality data from simulated model.

*The Roche model is built by F. Hoffmann-La Roche Ltd.*

In [1]:
# Load necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.stats import gamma, lognorm
import epimodels as em
import matplotlib
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the Roche Model

In [2]:
# Populate the model
total_days =  90
regions = ['London']
age_groups = ['0-1', '1-5', '5-15', '15-25', '25-45', '45-65', '65-75', '75+']

weeks = list(range(1,int(np.ceil(total_days/7))+1))
matrices_region = []

### Variable
# Initial state of the system
for w in weeks:
    weeks_matrices_region = []
    for r in regions:
        path = os.path.join('../../data/final_contact_matrices/{}_W{}.csv'.format(r, w))
        region_data_matrix = pd.read_csv(path, header=None, dtype=np.float64)
        regional = em.RegionMatrix(r, age_groups, region_data_matrix)
        weeks_matrices_region.append(regional)

    matrices_region.append(weeks_matrices_region)

contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
matrices_contact = [contacts]

# Matrices contact
time_changes_contact = [1]
time_changes_region = np.arange(1, total_days+1, 7).tolist()

### Fixed
# # Initial state of the system
# weeks_matrices_region = []
# for r in regions:
#     path = os.path.join('../../data/final_contact_matrices/BASE.csv')
#     region_data_matrix = pd.read_csv(path, header=None, dtype=np.float64)
#     regional = em.RegionMatrix(r, age_groups, region_data_matrix)
#     weeks_matrices_region.append(regional)

# matrices_region.append(weeks_matrices_region)

# contacts = em.ContactMatrix(age_groups, np.ones((len(age_groups), len(age_groups))))
# matrices_contact = [contacts]

# # Matrices contact
# time_changes_contact = [1]
# time_changes_region = [1]

# NPIs data
max_levels_npi = [3, 3, 2, 4, 2, 3, 2, 4, 2]
targeted_npi = [True, True, True, True, True, True, True, False, True]
general_npi = [
    [True, False, True, True, False, False, False, False, False],
    [True, False, True, True, True, True, False, False, False]]
time_changes_flag = [1, 12]

reg_levels_npi = [
    [[0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 3, 2, 4, 2, 3, 2, 4, 2]]]
time_changes_npi = [1, 14]

### Set the parameters and initial conditions of the model and bundle everything together

In [3]:
# Instantiate model
model = em.PheSEIRModel()

# Set the region names, age groups, contact and regional data of the model
model.set_regions(regions)
model.set_age_groups(age_groups)
model.read_contact_data(matrices_contact, time_changes_contact)
model.read_regional_data(matrices_region, time_changes_region)

# Initial number of susceptibles
path = os.path.join('../../data/england_population/England_population.csv')
total_susceptibles = np.loadtxt(path, dtype=int, delimiter=',').tolist()
susceptibles = total_susceptibles[-1]

# Initial number of infectives
ICs_multiplier = 30
infectives1 = (ICs_multiplier * np.ones((len(regions), len(age_groups)))).tolist()

infectives2 = np.zeros((len(regions), len(age_groups))).tolist()

# List of times at which we wish to evaluate the states of the compartments of the model
times = np.arange(1, total_days+1, 1).tolist()

# List of betas
betas = np.ones((len(regions), len(times))).tolist()

In [4]:
# Set regional and time dependent parameters
regional_parameters = em.PheRegParameters(
    model=model,
    initial_r=[2.35],
    region_index=1,
    betas=betas,
    times=times
)

# Set ICs parameters
ICs = em.PheICs(
    model=model,
    susceptibles_IC=[susceptibles],
    exposed1_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    exposed2_IC=np.zeros((len(regions), len(age_groups))).tolist(),
    infectives1_IC=infectives1,
    infectives2_IC=infectives2,
    recovered_IC=np.zeros((len(regions), len(age_groups))).tolist()
)

# Set disease-specific parameters
disease_parameters = em.PheDiseaseParameters(
    model=model,
    dL=4,
    dI=4
)

# Set other simulation parameters
simulation_parameters = em.PheSimParameters(
    model=model,
    delta_t=0.5,
    method='RK45'
)

# Set all parameters in the controller
parameters = em.PheParametersController(
    model=model,
    regional_parameters=regional_parameters,
    ICs=ICs,
    disease_parameters=disease_parameters,
    simulation_parameters=simulation_parameters
)

## Death and Serology data
### Read Death and Serology data

In [5]:
# Read in synthetic death, tests and positive data from external files
deaths_data = []
positives_data = []
tests = []

for region in regions:
    deaths_data.append(np.loadtxt('../../data/death_data/{}_deaths.csv'.format(region), dtype=int, delimiter=','))
    positives_data.append(np.loadtxt('../../data/serology_data/{}_positives_nhs.csv'.format(region), dtype=int, delimiter=','))
    tests.append(np.loadtxt('../../data/serology_data/{}_tests_nhs.csv'.format(region), dtype=int, delimiter=','))


In [6]:
# Select the time points for which the death and serology data is known
deaths_times = np.arange(1, total_days+1, 1).tolist()
serology_times = np.arange(1, total_days+1, 1).tolist()

In [7]:
# Set time-to-death using a Gamma distribution using the mean and standard deviation from the PHE paper
td_mean = 15.0
td_var = 12.1**2
theta = td_var / td_mean
k = td_mean / theta
time_to_death = gamma(k, scale=theta).pdf(np.arange(1, 31)).tolist()

# Set information
fatality_ratio = pd.read_csv('../../data/fatality_ratio_data/IFR.csv', usecols=['ifr'], dtype=np.float64)['ifr'].values.tolist()
time_to_death.extend([0.0] * (len(times)-30))
niu = 10**(-5) # float(gamma.rvs(1, scale=1/0.2, size=1))

sens = 0.7
spec = 0.95

## Sensitivity Analysis
### Observe changes in number of **new infections** and **deaths** for different ranges of parameters

In [8]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'black', 'green', 'purple', 'orange', 'gray', 'pink']

### 1. Varying _Initial R_

In [9]:
# Slider (initial_r)
titles = ['Infections', 'Deaths']
fig = go.Figure()
fig = make_subplots(rows=len(titles), cols=1, subplot_titles=tuple(titles), horizontal_spacing = 0.15)

# Add traces, one for each slider step
initial_r_range = np.arange(0.3, 5, 0.05)

for initial_r in initial_r_range:
    parameters.regional_parameters.initial_r = initial_r
    model_output = model.simulate(parameters)

    model_reg_deaths_data = np.empty(len(times))

    age_model_reg_new_infections = model.new_infections(model_output)
    model_reg_new_infections = age_model_reg_new_infections.sum(axis=1)

    for t, time in enumerate(times):
        model_reg_deaths_data[t] = np.sum(model.mean_deaths(fatality_ratio, time_to_death, t, age_model_reg_new_infections))
    
    predicted_new_infec = np.array(model_reg_new_infections)
    predicted_deaths = model_reg_deaths_data
    
    fig.add_trace(
        go.Bar(
            x=times,
            y=predicted_new_infec,
            name='Infections'
        ),
        row= 1,
        col= 1)

    fig.add_trace(
        go.Bar(
            x=times,
            y=predicted_deaths,
            name='Deaths'
        ),
        row= 2,
        col= 1)

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(int(len(fig.data)/2)):
    step = dict(
        method="restyle",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to initial_r: " + "{:.4f}".format(initial_r_range[i])}],  # layout attribute
        label = "{:.4f}".format(initial_r_range[i])
    )
    step["args"][0]["visible"][2 * i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][1 + 2 * i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Initial_r: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

# Add axis labels
fig.update_layout(
    width=600, 
    height=900,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis=dict(linecolor='black'),
    xaxis2=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis2=dict(linecolor='black')
)

fig.show()

In [10]:
# Slider (initial_r)
titles = ['Infections', 'Deaths']
fig = go.Figure()
fig = make_subplots(rows=len(titles), cols=1, subplot_titles=tuple(titles), horizontal_spacing = 0.15)

# Add traces, one for each slider step
initial_r_range = [0.5, 2.3, 3.28, 4]
traces = ['Initial R = {}'.format(r) for r in initial_r_range]
traces[2] += ' (BEST FIT)'

for r, initial_r in enumerate(initial_r_range):
    parameters.regional_parameters.initial_r = initial_r
    model_output = model.simulate(parameters)

    model_reg_deaths_data = np.empty(len(times))

    age_model_reg_new_infections = model.new_infections(model_output)
    model_reg_new_infections = age_model_reg_new_infections.sum(axis=1)

    for t, time in enumerate(times):
        model_reg_deaths_data[t] = np.sum(model.mean_deaths(fatality_ratio, time_to_death, t, age_model_reg_new_infections))
    
    predicted_new_infec = np.array(model_reg_new_infections)
    predicted_deaths = model_reg_deaths_data
    
    fig.add_trace(
        go.Scatter(
            x=times,
            y=predicted_new_infec,
            mode='lines',
            name=traces[r],
            line_color=colours[r]
        ),
        row= 1,
        col= 1)

    fig.add_trace(
        go.Scatter(
            x=times,
            y=predicted_deaths,
            mode='lines',
            showlegend=False,
            name=traces[r],
            line_color=colours[r]
        ),
        row= 2,
        col= 1)

fig.data[4].update(line=go.Line(width=3, dash='dot'))
fig.data[5].update(line=go.Line(width=3, dash='dot'))

# Add axis labels
fig.update_layout(
    width=600, 
    height=900,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis=dict(linecolor='black'),
    xaxis2=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis2=dict(linecolor='black'),
    legend=dict(
        orientation='h',
        yanchor="bottom",
        y=1.075,
        xanchor="right",
        x=1)
)

fig.write_image('images/Phe_Sensitivity_R0.pdf')
fig.show()

/Users/ioaros/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




### 2. Varying _sigma beta_

In [11]:
parameters.regional_parameters.initial_r = [3.28]

# Slider (beta_multiplier)
titles = ['Infections', 'Deaths']
fig = go.Figure()
fig = make_subplots(rows=len(titles), cols=1, subplot_titles=tuple(titles), horizontal_spacing = 0.15)

# Add traces, one for each slider step
beta_multiplier_range = np.arange(0.5, 1.65, 0.05)

for beta_multiplier in beta_multiplier_range:
    new_betas = beta_multiplier * np.array(betas)
    parameters.regional_parameters.betas = new_betas.tolist()
    
    model_output = model.simulate(parameters)

    model_reg_deaths_data = np.empty(len(times))

    age_model_reg_new_infections = model.new_infections(model_output)
    model_reg_new_infections = age_model_reg_new_infections.sum(axis=1)

    for t, time in enumerate(times):
        model_reg_deaths_data[t] = np.sum(model.mean_deaths(fatality_ratio, time_to_death, t, age_model_reg_new_infections))
    
    predicted_new_infec = np.array(model_reg_new_infections)
    predicted_deaths = model_reg_deaths_data
    
    fig.add_trace(
        go.Bar(
            x=times,
            y=predicted_new_infec,
            name='Infections'
        ),
        row= 1,
        col= 1)

    fig.add_trace(
        go.Bar(
            x=times,
            y=predicted_deaths,
            name='Deaths'
        ),
        row= 2,
        col= 1)

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(int(len(fig.data)/2)):
    step = dict(
        method="restyle",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to beta_multiplier: " + "{:.4f}".format(beta_multiplier_range[i])}],  # layout attribute
        label = "{:.4f}".format(beta_multiplier_range[i])
    )
    step["args"][0]["visible"][2 * i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][1 + 2 * i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Beta_multiplier: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

# Add axis labels
fig.update_layout(
    width=600, 
    height=900,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis=dict(linecolor='black'),
    xaxis2=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis2=dict(linecolor='black')
)

fig.show()

In [12]:
# Slider (beta_multiplier)
titles = ['Infections', 'Deaths']
fig = go.Figure()
fig = make_subplots(rows=len(titles), cols=1, subplot_titles=tuple(titles), horizontal_spacing = 0.15)

# Add traces, one for each slider step
beta_multiplier_range = [0.6, 1, 1.3, 1.7]
traces = ['Multiplier β = {}'.format(b) for b in beta_multiplier_range]
traces[1] += ' (BEST FIT)'

for b, beta_multiplier in enumerate(beta_multiplier_range):
    new_betas = beta_multiplier * np.array(betas)
    parameters.regional_parameters.betas = new_betas.tolist()
    
    model_output = model.simulate(parameters)

    model_reg_deaths_data = np.empty(len(times))

    age_model_reg_new_infections = model.new_infections(model_output)
    model_reg_new_infections = age_model_reg_new_infections.sum(axis=1)

    for t, time in enumerate(times):
        model_reg_deaths_data[t] = np.sum(model.mean_deaths(fatality_ratio, time_to_death, t, age_model_reg_new_infections))
    
    predicted_new_infec = np.array(model_reg_new_infections)
    predicted_deaths = model_reg_deaths_data
    
    fig.add_trace(
        go.Scatter(
            x=times,
            y=predicted_new_infec,
            mode='lines',
            name=traces[b],
            line_color=colours[b]
        ),
        row= 1,
        col= 1)

    fig.add_trace(
        go.Scatter(
            x=times,
            y=predicted_deaths,
            mode='lines',
            showlegend=False,
            name=traces[b],
            line_color=colours[b]
        ),
        row= 2,
        col= 1)

fig.data[2].update(line=go.Line(color='black', width=3, dash='dot'))
fig.data[3].update(line=go.Line(color='black', width=3, dash='dot'))

fig.data[4].update(line=go.Line(color='red'))
fig.data[5].update(line=go.Line(color='red'))

# Add axis labels
fig.update_layout(
    width=600, 
    height=900,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis=dict(linecolor='black'),
    xaxis2=dict(
        linecolor='black',
        tickvals=np.arange(1, total_days, 10).tolist(),
        ticktext=['Feb 15', 'Feb 25', 'Mar 06', 'Mar 16', 'Mar 26', 'Apr 05', 'Apr 15', 'Apr 25', 'May 05', 'May 15', 'May 25', 'Jun 04', 'Jun 14', 'Jun 24']),
    yaxis2=dict(linecolor='black'),
    legend=dict(
        orientation='h',
        yanchor="bottom",
        y=1.075,
        xanchor="right",
        x=1)
)

fig.write_image('images/Phe_Sensitivity_SigmaB.pdf')
fig.show()